### This notebook analyzes the orientation histograms obtained by the Fiji script (analyze_fiber_orientation.py)

The notebook is split into two parts. Both parts can be run independently.
#### Part 1: analysis of a single dataset
* Generated output:
    * Figures with multiple histograms of different slices, with different normalization options
    * Figure with average coherency per slice
    * The plots are automatically saved into the folder of the input data.

#### Part 2: combine multiple datasets and get statistics
* For each dataset the slice with the strongest horizontal resp. longitudinal fiber orientation is detected. These curves are then averaged and the standard deviation is computed.
* Generated output:
    * Figure with average+-stddev of orientation distribution of longitudinal and circular fibers.
    * Additionally an illustrative figure of chosen fiber distributions per dataset.
    * The plots are automatically saved into a user-defined folder.

#### Prerequisites:
* The Fiji script analyze_fiber_orientation.py was run on a dataset.
* For part 2: multiple datasets were analyzed.
* This notebook uses the Orientation Histogram data stored as orientationDistribution_CoherencyWeighted.csv, resp. orientationDistribution_Unweighted.csv. This data was automatically generated during fiber analysis in Fiji and should not be renamed.

#### General Jupyter Notebook Usage: The basics
* The notebook is executed cell-by-cell.
* The active/current cell is visible by being surrounded by a blue or green box.
* Green box: edit mode. Blue box: view mode. To exit the edit mode: press Esc
* To run a cell: Press Shift+Enter (runs the cell and automatically advances to the next cell)
* Auto-complete is supported (convenient when you e.g. type a folder location): start typing, then press Tab key.
* When you are completely finished: Click on Kernel->Shutdown and close all notebooks related tabs.
* If you want to do a fresh start of the analysis: click on Kernel -> Restart and Clear Output

#### Usage of this notebook:
* Start at part 1 and/or part 2.
* Run cells step-by-step.
* Some cells are titled "Adjust". Adjust the content (data folder etc.) before running them. 
    * Note: if you accidentally ran it already, simply click on it, adjust the content, and run it again.   
    
#### Good to know:
* part 2 assumes that the planarian is vertically aligned.


#### Changelog
* 2019-07-18: added part 2: combine multiple datasets
* 2019-06-11: added figure with normalized orientation distribution

===================================================================================================

### Preparations: always run this cell.

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from helpers_histogram_analysis import *

<br>

## Part 1: analysis of a single dataset



### This cell: Adjust

In [ ]:
# == folder where your final .csv data resides (typically "/some/path/5_orientation_analysis/data") ==
data_folder="/Users/walker/Projects/OrientationMuscleFibers_2283/repo_public/example_data/results/testdata/5_orientation_analysis/data/"

# == weighted or unweighted histogram (weighted=weighted by coherency). == 
# Recommended: use_weighted=True
use_weighted=True # options: True, False

### Automatic: Load the csv data and display a few lines

In [ ]:
fn_load=get_filename_csv_histdata(data_folder, use_weighted)

df=pd.read_csv(fn_load)
display(df.head())

# convert to numpy. remove the "sliceX" string label. first row: slice1, second rw: slice2 etc
data=(df.iloc[:,1:]).values

<br>

### This cell: Adjust. Specify the details of the histogram plot. 

In [ ]:
# == choose slices to be plotted ==
# options: None (uses all), or a list with slice numbers (counting starts at 1), e.g. [8,10,12]
slice_selection=None
#slice_selection=[10,22,30]

# == choose color map: every string from here https://matplotlib.org/tutorials/colors/colormaps.html will work. ==
# careful: some color ranges extend to white -> invisible.
# recommended: None (will use a default), "jet", "gnuplot", "copper"
color_map=None

### Automatic: plots the orientation histogram and the coherency
#### un-normalized distribution

In [ ]:
plot_hist(data,slice_selection,use_weighted,False,data_folder, color_map, ylim=None)

#### distribution normalized to max=1 (per slice)

In [ ]:
plot_hist(data,slice_selection,use_weighted,True,data_folder, color_map, ylim=None)

#### average coherency per slice

In [ ]:
plot_coherency(data,use_weighted,data_folder)

<br>

## Part 2: Combine multiple datasets and get statistics
### This cell: Adjust

**data_dir_list:** <br>
Specify a list of folders where your final .csv files reside. The folders should belong to replicates of the same experiment. <br>
Example (be careful with the [] and commas):
>data_dir_list=[<br>
"/experiment_1/5_orientation_analysis/data",<br>
"/experiment_2/5_orientation_analysis/data",<br>
"/experiment_3/5_orientation_analysis/data"<br>
]

**save_dir:** <br>
Folder where to save the figures. Will be created if the path does not exist.

In [ ]:
data_dir_list=[
    "/Users/walker/Projects/OrientationMuscleFibers_2283/enhancements/2019-07-histogram_averaging/data/EGFP_ventralsides/data_1_1/",
    "/Users/walker/Projects/OrientationMuscleFibers_2283/enhancements/2019-07-histogram_averaging/data/EGFP_ventralsides/data_1_2/",
    "/Users/walker/Projects/OrientationMuscleFibers_2283/enhancements/2019-07-histogram_averaging/data/EGFP_ventralsides/data_2_3/",
    "/Users/walker/Projects/OrientationMuscleFibers_2283/enhancements/2019-07-histogram_averaging/data/EGFP_ventralsides/data_2_4/"
]

save_dir="/Users/walker/Projects/JamesHanh_OrientationMuscleFibers_2283/tmp/"

### Automatic: Get statistics of combined datasets
#### Load all datasets and find slices with highest peak

In [ ]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

use_weighted=True # coherency weighted

datasets,sliceids_circ,sliceids_long=load_datasets_and_find_peaks(data_dir_list,use_weighted,
       peak_circular_expected=0,peak_longitudinal_expected=90) # expected peak locations in degree, between [-90,90]

#### Show the selected slices

In [ ]:
titles=["dataset "+str(x) for x in range(len(data_dir_list))]
quick_plot_of_peak_selection(datasets,use_weighted,sliceids_long,sliceids_circ,titles,save_dir=save_dir,ylim=None)

#### Final result: Compute statistics and plot them

In [ ]:
ymean_circ, ystddev_circ, ymean_long, ystddev_long=get_statistics(datasets,sliceids_long,sliceids_circ)

plot_average_histogram(ymean_circ,ystddev_circ,ymean_long,ystddev_long, numsamples=len(datasets),
                       save_dir=save_dir, ylim=None) # use e.g. ylim=[0,0.07] for fixed y-range